Metode TOPSIS : metode pengambilan keputusan multikriteria dengan ide dasarnya alternatif yang dipilih memiliki jarak terdekat dengan solusi ideal positif dan memiliki jarak terjauh dari solusi ideal negatif.

In [17]:
import pandas as pd
import numpy as np

In [18]:
path = "dataset/dataset_clean.csv"
df = pd.read_csv(path)
# df = df.head(4)
print(df)

                         Corporation  ROE - Return On Equity  \
0          American States Water Co.                  8.1724   
1     Automatic Data Processing Inc.                 22.0354   
2                         Avnet Inc.                 13.6376   
3       California Water Service Co.                  9.6412   
4               Cardinal Health Inc.                 11.1256   
...                              ...                     ...   
1372                   Welbilt, Inc.                 12.9974   
1373         Westlake Chemical Corp.                 18.6697   
1374                     Xerox Corp.                  9.7833   
1375            YPF Sociedad Anonima                  3.6742   
1376       iHeartCommunications Inc.                 11.8335   

      Net Profit Margin  Operating Cash Flow Per Share  Debt/Equity Ratio  
0                8.3224                         1.9957             0.8847  
1               13.5690                         0.2501             0.0073  
2  

Normaliasi

In [19]:
def normalize_topsis(matrix):
    return matrix / np.sqrt(np.sum(matrix**2, axis=0))

Fungsi perhitungan dengan topsis

In [20]:
def topsis_method(matrix, weights, benefit_criteria):
    # Normalisasi matrix
    normalized = normalize_topsis(matrix)
    
    # Mengalikan dengan bobot
    weighted = normalized * weights
    
    # Menentukan solusi ideal positif dan negatif
    ideal_best = np.max(weighted, axis=0)
    ideal_worst = np.min(weighted, axis=0)
    
    # Untuk kriteria cost, tukar nilai ideal best dan worst
    for i in range(len(benefit_criteria)):
        if i not in benefit_criteria:
            ideal_best[i], ideal_worst[i] = ideal_worst[i], ideal_best[i]
    
    # Menghitung jarak ke solusi ideal
    s_best = np.sqrt(np.sum((weighted - ideal_best)**2, axis=1))
    s_worst = np.sqrt(np.sum((weighted - ideal_worst)**2, axis=1))
    
    # Menghitung skor
    topsis_score = s_worst / (s_best + s_worst)
    
    return topsis_score

Menentukan Kriteria, bobot dan benefit

In [21]:
criteria = ['ROE - Return On Equity', 'Net Profit Margin', 'Operating Cash Flow Per Share', 'Debt/Equity Ratio']
weights = np.array([0.30, 0.25, 0.20, 0.25])
benefit_criteria = [0, 1, 2]  # Indeks kolom untuk kriteria benefit (0-based)

Membuat Matrix dari data 

In [22]:
matrix = df[criteria].values

Menghitungung skor TOPSIS

In [23]:
# Menghitung skor TOPSIS
scores = topsis_method(matrix, weights, benefit_criteria)

# Menambahkan skor ke dataframe
df['Score'] = scores

# Mengurutkan berdasarkan skor
df_sorted = df.sort_values('Score', ascending=False).reset_index(drop=True)

print("Hasil Perangkingan:")
print(df_sorted[['Corporation', 'Score'] + criteria])

# Rekomendasi
print("\nRekomendasi:")
print(f"Berdasarkan analisis TOPSIS, perusahaan {df_sorted.iloc[0]['Corporation']} memiliki skor tertinggi dan merupakan pilihan terbaik untuk investasi.")

# Analisis tambahan
print("\nAnalisis Perbandingan:")
best_company = df_sorted.iloc[0]
for i, row in df_sorted.iterrows():
    if i == 0:
        continue
    comparison = row['Score'] / best_company['Score']
    print(f"Perusahaan {row['Corporation']} memiliki {comparison:.2%} dari kinerja perusahaan terbaik ({best_company['Corporation']}).")

Hasil Perangkingan:
                                        Corporation     Score  \
0                       LOCKHEED MARTIN CORPORATION  0.712587   
1                            Nabors Industries Inc.  0.694724   
2                             Laredo Petroleum Inc.  0.610725   
3                            Laredo Petroleum, Inc.  0.610725   
4                               LIBERTY MEDIA CORP.  0.606249   
...                                             ...       ...   
1372                 Pacific Life Insurance Company  0.520910   
1373  Global One Real Estate Investment Corporation  0.519586   
1374                      Eldorado Gold Corporation  0.517240   
1375                  Commercial Vehicle Group Inc.  0.300687   
1376                              Pitney Bowes Inc.  0.197727   

      ROE - Return On Equity  Net Profit Margin  \
0                  7038.4610             5.8179   
1                     2.5612            -2.4218   
2                    13.9440            20.846